### Dados futebol: https://www.kaggle.com/hugomathien/soccer

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
caminho_banco = './data/soccer.sqlite'
conn = sqlite3.connect(caminho_banco)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

OperationalError: unable to open database file

In [ ]:
countries = pd.read_sql("""SELECT * FROM Country;""", conn)
countries

In [ ]:
leagues = pd.read_sql("""SELECT * FROM League 
                        JOIN Country 
                        ON Country.id = League.country_id;""", conn)
leagues

In [ ]:
league_df = pd.read_sql("SELECT * FROM League", conn)
countries_df = pd.read_sql("SELECT * FROM Country", conn)


In [ ]:
print("Leagues")
print(league_df.head(2))
print("Countries")
print(countries_df.head(2))

In [ ]:
league_df.merge(countries_df, 
                left_on="country_id", 
                right_on="id", 
                suffixes=["_league", "_countries"] )

In [ ]:
teams = pd.read_sql("""SELECT * FROM Team 
                        ORDER BY team_long_name 
                        LIMIT 10;""", conn)
teams

# Uma query um pouco mais complexa:


In [ ]:
leages_by_season = pd.read_sql("""
        SELECT 
                season, 
                COUNT(Match.id), 
                AVG(home_team_goal), 
                AVG(away_team_goal), 
                AVG(home_team_goal + away_team_goal) AS MediaGoalsNaPartida , 
                AVG(home_team_goal - away_team_goal) 
        FROM MATCH
        JOIN Country on Country.id=Match.country_id
        JOIN League on League.id = Match.league_id 
        LEFT JOIN Team AS HomeTeam on HomeTeam.team_api_id = Match.home_team_api_id
        LEFT JOIN Team As AwayTeam on AwayTeam.team_api_id = Match.away_team_api_id
        WHERE Country.name = "England"
        GROUP BY season;""", conn)
leages_by_season

 # Merging os DataFrames

In [ ]:
countries_df = pd.read_sql("SELECT * FROM Country", conn)
print("Countries shape: {}".format(countries_df.shape))
print("Columns : {}".format(countries_df.columns))
leagues_df = pd.read_sql("SELECT * FROM League", conn)
print("Leagues shape: {}".format(leagues_df.shape))
print("Columns : {}".format(leagues_df.columns))
team_df = pd.read_sql("SELECT * FROM Team", conn)
print("Team shape: {}".format(team_df.shape))
print("Columns : {}".format(team_df.columns))
matches_df = pd.read_sql("SELECT * FROM Match", conn)
print("Matches shape: {}".format(matches_df.shape))
print("Columns : {}".format(matches_df.columns))


In [ ]:
matches_countries_df = matches_df.merge(countries_df,
                                        left_on="country_id", 
                                        right_on="id", 
                                        suffixes=["_match","_country"])
print(matches_countries_df.shape)

In [ ]:
matches_countries_leagues_df = matches_countries_df.merge(leagues_df, 
                                                          left_on="league_id", 
                                                          right_on="id", 
                                                          suffixes=["_country","_league"])
print(matches_countries_leagues_df.shape)

In [ ]:
hometeam_matches_countries_leagues_df = matches_countries_leagues_df.merge(
    team_df, how="left",
    left_on="home_team_api_id", 
    right_on="team_api_id")

away_home_team_matches_countries_leagues_df = hometeam_matches_countries_leagues_df.merge(
    team_df, how="left",
    left_on="away_team_api_id", 
    right_on="team_api_id", 
    suffixes=["_home","_away"])

print(away_home_team_matches_countries_leagues_df.shape)
# awayteam_matches_countries_leagues_df["team_short_name_away"]

In [ ]:
# away_home_team_matches_countries_leagues_df[["season","match_api_id","team_long_name_home","team_long_name_away"]]


In [ ]:
england_df = away_home_team_matches_countries_leagues_df[
                    away_home_team_matches_countries_leagues_df["name_country"] == "England"
    ]

In [ ]:
away_team_goal_mean = england_df.groupby("season").mean()["away_team_goal"]
away_team_goal_mean

In [ ]:
operations = {
    "id_match" : "count",
    "home_team_goal" : "mean",
    "away_team_goal" : "mean"
}
stats_df= england_df.groupby("season").agg(operations)
print(stats_df)

In [ ]:
stats_df["media_goals_na_partida"] = stats_df["away_team_goal"] + stats_df["home_team_goal"]
stats_df["home_diff_away"] = stats_df["home_team_goal"] - stats_df["away_team_goal"]
print(stats_df)

In [ ]:
ax = stats_df["home_diff_away"].plot()
ax.set_ylim(0,1)